In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

C:\Users\mook\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
x_train = mnist.train.images[:,:]
y_train = mnist.train.labels[:,:]

In [3]:
x_test = mnist.test.images[:,:]
y_test = mnist.test.labels[:,:]

# CNN+DNN+softmax+xavier

In [4]:
# 프로젝트 CNN  Layer 구성하기 
# Condition
# 1층 : 3*3 필터 32개 (dropout rate: 0.3 )
# relu-max pooling
# 2층 : 3*3 필터 64개 (dropout rate: 0.3 )
# relu-max pooling
# 3층 : 3*3 필터 128개 (dropout rate: 0.3 )
# relu-max pooling

# DNN(FC)  Layer
# Initializer : xavier initializer
# 1층 : hidden node 625개 (dropout rate: 0.3 )
# activation function : Relu
# 2층 : hidden node 256개 (dropout rate: 0.3 )

# Softmax Layer로 multi label classification
# AdamOptimizer
# learning_rate : 0.001

In [5]:
learning_rate = 0.01
training_epoch = 15
batch_size =100

In [6]:
# CNN에서 이미지를 전처리하고 DNN에서 구현한다 - CNN은 독립적으로 사용하지 않음
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

# CNN에 이미지를 넣으면 data를 image로 바꿔야 한다
X_img = tf.reshape(X, [-1, 28, 28, 1])  
W1 = tf.get_variable("W1", shape=[3, 3, 1, 32], initializer=tf.contrib.layers.xavier_initializer())  # weight 변수 지정
b1 = tf.Variable(tf.random_normal([32]))   # convol (filter) layer
L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = "SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")  # pooling
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)   # dropout = 30% 제외

W2 = tf.get_variable("W2", shape=[3, 3, 32, 64], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = "SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[3, 3, 64, 128], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.conv2d(L2, W3, strides = [1,1,1,1], padding = "SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)   # dropout은 다음 층으로 넘어가기 직전에 실시

L3_flat = tf.reshape(L3, [-1, 128*4*4])   # SAME = 28 / 2 = 14 / 2 = 7 / 2 = 4

W4 = tf.get_variable("W4", shape=[128*4*4, 625], initializer=tf.contrib.layers.xavier_initializer()) 
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4) 
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[625, 256], initializer=tf.contrib.layers.xavier_initializer()) 
b5 = tf.Variable(tf.random_normal([256]))
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5) 
L5 = tf.nn.dropout(L5, keep_prob=keep_prob)

W6 = tf.get_variable("W6", shape=[256, 10], initializer=tf.contrib.layers.xavier_initializer()) 
b6 = tf.Variable(tf.random_normal([10]))

logits = tf.matmul(L5, W6) + b6
hypothesis = tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)  
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)  
        c, _ = sess.run([cost, optimizer], feed_dict={X:batch_xs, Y:batch_ys})
        avg_cost += c/total_batch 
    print("Epoch: ", epoch, "Cost: ", avg_cost) 

In [ ]:
print(sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))